## Se utilizara la base de datos vectorial de Pinecone

### Procesamiento de documentos

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
# Genero los chunks
def chunkData(docs, chunk_size=100, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(docs)
    return chunks

Uso 2 CVs, el de mi traumatologo y el mio

In [3]:
# Cargo los documentos 
miCV = "./CV_CristianMarino_CEIA_LLM.pdf"
jorgeCV = "./CV-JorgeBoretto.pdf"

cloader = PyPDFLoader(miCV)
cdocs = cloader.load()

jloader = PyPDFLoader(jorgeCV)
jdocs = jloader.load()

# Genero los chunks 
cchunks = chunkData(cdocs, chunk_size=500, chunk_overlap=100)
jchunks = chunkData(jdocs, chunk_size=500, chunk_overlap=100)

Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)


### Generación de embbeding y carga en base de datos vectorial
Se utilizara un indice por cada uno de los cv

In [4]:
import os
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

from dotenv import load_dotenv

In [5]:
load_dotenv(dotenv_path='1.env')
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")

In [6]:
#Connect to DB Pinecone
pc=Pinecone(api_key=PINECONE_API_KEY)

cloud = 'aws'
region = 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

indices = ['cagent', 'jagent']
namespace = "espacio"
dimension = 384

In [7]:
pc.list_indexes().names()

['ragllm', 'tp2llm', 'jagent', 'cagent']

In [8]:
# Elimino el indice si es que ya existe en la base de datos
for index_name in indices:
    if index_name in pc.list_indexes().names():
      pc.delete_index(index_name)
      print("index {} borrado".format(index_name))
    
    if index_name not in pc.list_indexes().names():
        # Como lo borre en el paso anterior siempre deberia entrar aca
        print("index creado con el nombre: {}".format(index_name))
        pc.create_index(
            index_name,
            dimension=dimension, 
            metric='cosine',
            spec=spec
            )
    else:
        print("el index con el nombre {} ya estaba creado".format(index_name))
    

index cagent borrado
index creado con el nombre: cagent
index jagent borrado
index creado con el nombre: jagent


In [9]:
# Cargo un modelo de embeddings compatible
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Intel Core I5 10400\AppData\Local\Temp\ipykernel_10780\1365815625.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
# Cargo chunks en base de datos
docsearch = PineconeVectorStore.from_documents(
    documents=cchunks,
    index_name='cagent',
    embedding=embedding_model, 
    namespace=namespace
)
print("upserted values to 'cagent' index")

time.sleep(1)

upserted values to 'cagent' index


In [11]:
# Cargo chunks en base de datos
docsearch = PineconeVectorStore.from_documents(
    documents=jchunks,
    index_name='jagent',
    embedding=embedding_model, 
    namespace=namespace
)
print("upserted values to 'jagent' index")

time.sleep(1)

upserted values to 'jagent' index


### Busquedas en base de datos

Realizo pruebas para verificar que los datos se guardaron correctamente

## CRISTIAN

In [12]:
index = pc.Index(indices[0])
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'espacio': {'vector_count': 9}},
 'total_vector_count': 9}

In [13]:
vectorstore = PineconeVectorStore(
    index_name=indices[0],
    embedding=embedding_model,
    namespace=namespace,
)
retriever=vectorstore.as_retriever()

In [14]:
query = "Qué hace en Fecovita"
vectorstore.similarity_search(query, k=3)

[Document(id='02a1ebc7-30b3-4e0f-adcc-d8d30ccfe349', metadata={'page': 0.0, 'source': './CV_CristianMarino_CEIA_LLM.pdf'}, page_content='PYTHON                                  \nPOWER BI                                \nSenior Project Manager                                         Oct 2023 - Today\nFECOVITA| Argentina\nPMO Leader. Responsible for CAPEX portfolio management.\nFormulation, feasibility study, bidding, execution and final audit of projects,\nPresentation of Investment opportunities and funding proposals to the Board.\nAchievements: 5 pre-approved projects. Currently, under bidding,\nWORK EXPERIENCE'),
 Document(id='ccde11a9-8292-4282-9078-7ae674e80d5b', metadata={'page': 0.0, 'source': './CV_CristianMarino_CEIA_LLM.pdf'}, page_content='cristian.dam.marino@gmail.com\n+ 54 9 2616504324\nMendoza, Argentina\nLANGUAGES AND TECH SKILLS\nENGLISH                                                  (C1)\nPORTUGUESE                                        (B2)\nFRENCH                 

#### Jorge Boretto

In [15]:
index = pc.Index(indices[0])
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'espacio': {'vector_count': 9}},
 'total_vector_count': 9}

In [16]:
jvectorstore = PineconeVectorStore(
    index_name=indices[1],
    embedding=embedding_model,
    namespace=namespace,
)
retriever=jvectorstore.as_retriever()

In [17]:
query = "Cuales es su educacion?"
jvectorstore.similarity_search(query, k=3)

[Document(id='85558b05-ed92-40b4-a4a4-50b9e7252f8a', metadata={'page': 26.0, 'source': './CV-JorgeBoretto.pdf'}, page_content='y Matemáticas. Secretaría de Graduados en Ciencias de la Salud, Universidad Nacional de Córdoba. Duración 20 hs. (aprobado). Córdoba, Argentina.  IDIOMA  Francés - 05.2003. Curso de Francés Aprobado. Cours de Langue 1 de Français Langue Etrangère. Alliance Française de Córdoba. Duración 39 hs. Córdoba, Argentina.  Ingles - 12.2002. Examen de Inglés Aprobado. Secretaría de Graduados en Ciencias de la Salud, Universidad Nacional de Córdoba. Córdoba, Argentina. - 03.1999. Educación Continua de'),
 Document(id='91475b1f-a390-408b-b915-ff60967f46fb', metadata={'page': 9.0, 'source': './CV-JorgeBoretto.pdf'}, page_content='para Cobertura de Grandes Defectos en Mano”. 38º Congreso Argentino de Cirugía de la Mano. Mar del Plata, Argentina.  AÑO 2011 ü “Colgajos en Isla de la Mano”. Simposio: “Colgajos en el Miembro Superior”. 48° Congreso Argentino de Ortopedia y Traum

## Agente LLM

In [18]:
pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
from groq import Groq

from langchain.chains import ConversationChain, LLMChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langgraph.graph import START, StateGraph, END
from langchain import hub
from langchain.prompts import PromptTemplate

import re

In [20]:
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

In [21]:
llm = ChatGroq(
    groq_api_key=GROQ_API_KEY, 
    model_name='llama3-8b-8192'
)

In [23]:
# Defino una clase para guardar el estado 
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str
    individual: str
    history: List[str] 

# Defino un tamplate para el prompt
prompt = PromptTemplate(
    input_variables=["context", "question", "individual"],
    template="""
Eres un asistente para tareas de preguntas y respuestas. Usa los siguientes fragmentos de historia y contexto recuperados para responder la pregunta respecto al individuo.
Si no sabes la respuesta, di que no lo sabes. Usa un máximo de 200 palabras y mantén la respuesta concisa. 
---
Historia:
{history}
---
Contexto:
{context}
---
Individuo:
{individual}
---
Pregunta: {question}
Respuesta:
"""
)


# Defino una clase agente para hacer la busqueda en la base vectorial segun la persona
class Agent:
    
    def __init__(self, embedding_model, index=""):
        if index=="":
            raise ValueError("No se especifico un índice válido.")
        
        self.index = index
        self.embedding_model = embedding_model

        self.vectorstore = PineconeVectorStore(
            index_name=index,
            embedding=self.embedding_model,
            namespace=namespace,
        )

    def get_context(self,state: State):
        retrieved_docs = self.vectorstore.similarity_search(state["question"],k=2)
        return {"context": retrieved_docs}

In [24]:
# Instancio agentes
cagent = Agent(embedding_model,"cagent")
jagent = Agent(embedding_model,"jagent")

In [25]:
# Defino los nodos para el agente
def generate(state: State):
    if state["context"]:
        docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    else: 
        docs_content = ""
    # Formateo la historia como un unico string
    history = "\n".join(state["history"])
    
    # Invoco el prompt con contexto e historia previa
    messages = prompt.invoke({
        "question": state["question"],
        "context": docs_content,
        "individual": state["individual"],
        "history": history
    })

    # print(messages)
    response = llm.invoke(messages)
    
    state["history"].append(f"Q: {state['question']} A: {response.content}")
    
    # Ahora ya es posible devolver la respuesta
    return {"answer": response.content}

# Nodo para limpiar el contexto
def empty_context(state:State):
    return {"context":[]}

# Segun sobre a quien se refiere la pregunta se utiliza un agente u otro
def decide(state: State):
    
    cristian_pattern = r"(Cristian\sMarino|Cristian|Marino)"
    jorge_pattern = r"(Jorge\sBoretto|Jorge|Boretto)"
    individual = "" 
    
    if re.search(cristian_pattern, state["question"], re.IGNORECASE):
        individual = "cristian"
    elif re.search(jorge_pattern, state["question"], re.IGNORECASE):
        individual = "jorge"
    return {"individual":individual}

# Funcion para determinar cuál es el próximo nodo
def decision_read_state(state:State):
    """Obtiene el individuo desde el state y lo retorna para decidir por qué nodo continuar."""
    indiv = state["individual"]
    if indiv=="":
        print("La pregunta no habla de ningun individuo.")
        return "no_individual"
    print("La pregunta habla sobre el individuo:",indiv)
    return indiv

In [26]:
# Armo el grafo de nodos
graph_builder = StateGraph(State)
graph_builder.add_node("decision",decide)
graph_builder.add_node("cristian_agent",cagent.get_context)
graph_builder.add_node("jorge_agent",jagent.get_context)
graph_builder.add_node("generate",generate)
graph_builder.add_conditional_edges(
    "decision",
    decision_read_state,
    {"cristian": "cristian_agent","jorge": "jorge_agent","no_individual":"cristian_agent"}
    )
graph_builder.add_edge("cristian_agent","generate")
graph_builder.add_edge("jorge_agent","generate")
graph_builder.set_entry_point("decision")
graph = graph_builder.compile()

## No me sale usar graph viz, ni json, ni networkx

In [27]:
def convert_to_dot(graph):
    dot_representation = 'digraph G {\n'
    for node_id, node in graph.nodes.items():
        dot_representation += f'    "{node_id}"\n'
    for edge in graph.edges:
        if edge.conditional:
            dot_representation += f'    "{edge.source}" -> "{edge.target}" [label="{edge.data}"]\n'
        else:
            dot_representation += f'    "{edge.source}" -> "{edge.target}"\n'
    dot_representation += '}'
    return dot_representation

In [28]:
# Este formato lo utilizo en la pagina web para generar el grafico
print(convert_to_dot(graph.get_graph()))

digraph G {
    "__start__"
    "decision"
    "cristian_agent"
    "jorge_agent"
    "generate"
    "__start__" -> "decision"
    "cristian_agent" -> "generate"
    "jorge_agent" -> "generate"
    "decision" -> "cristian_agent" [label="cristian"]
    "decision" -> "jorge_agent" [label="jorge"]
    "decision" -> "cristian_agent" [label="no_individual"]
}


### Prueba del agente

In [30]:
response = graph.invoke({"question": "Jorge es médico?","history":[]})
print(response["answer"])

La pregunta habla sobre el individuo: jorge
Sí, Jorge Boretto es médico, según se menciona en su perfil que trabaja en la Clínica de la Mano de Buenos Aires.


In [31]:
response = graph.invoke({"question": "Contame sobre la experiencia de Cristian","history":[]})
print(response["answer"])

La pregunta habla sobre el individuo: cristian
Cristian tiene experiencia en gestión general de legal, financiera y de auditoría, y ha liderado 6 equipos. Ha trabajado con múltiples stakeholders y ha sido representante de la oficina chilena ante el Consejo Financiero de la Sede Central. Además, ha logrado varios logros notables, como la renegociación de 5 contratos clave que llevó a un crecimiento del 28% en la renta y un aumento de los reservas financieras en un 450%. También ha reducido el tiempo de cobro en 23 días.


In [32]:
response = graph.invoke({"question": "Dónde trabaja?","history":[]})
print(response["answer"])

La pregunta no habla de ningun individuo.
No tengo información sobre el lugar de trabajo actual del individuo. Sin embargo, puedo mencionar que en el pasado fue VP de Finance and Legal en The AIESEC Foundation en Chile, desde agosto de 2019 hasta julio de 2020.
